In [1]:
import os
import time

import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, SequentialSampler, ConcatDataset

from chord_recognition.dataset import AudioDataset
from chord_recognition.metrics import compute_eval_measures
from chord_recognition.cnn import model
from chord_recognition.predict import predict_annotations, model, device

In [2]:
# chord_dataset = AudioDataset(audio_dir="data/robbie_williams/mp3/",
#                              ann_dir="data/robbie_williams/chordlabs/",
#                             window_size=8192, hop_length=4096)
chord_dataset = AudioDataset(audio_dir="data/queen/mp3/",
                             ann_dir="data/queen/chordlabs/",
                             window_size=8192, hop_length=4096)
queen = AudioDataset(audio_dir="data/beatles/mp3s-32k/",
                     ann_dir="data/beatles/chordlabs/",
                     window_size=8192, hop_length=4096)
chord_dataset = ConcatDataset([chord_dataset, queen])
loader_chord = DataLoader(chord_dataset, batch_size=None, num_workers=0)
sampler = SequentialSampler(chord_dataset)

In [3]:
def estimate_chords(dataloader, model, device, batch_size=32,
                    drop_last=False):
    model.eval()  # set model to evaluation mode
    total_R = total_P = total_F = 0.0
    count = 0
    
    for idx in dataloader:
        sample = dataloader.data_source[idx]
        sample_name = sample['sample']
        chromagram = sample['chromagram']
        
        result = predict_annotations(chromagram, model, device, batch_size=batch_size)
        ann_matrix = sample['ann_matrix']
        count += 1
        
        P, R, F1, TP, FP, FN = compute_eval_measures(ann_matrix, result)
        total_R += R
        total_P += P
        total_F += F1
        title = f'Eval: (N=%d, TP=%d, FP=%d, FN=%d, P=%.3f, R=%.3f, F=%.3f) - {sample_name}' % (result.shape[1], TP, FP, FN, P,R,F1)
        print(title)
    total_R /= count
    total_P /= count
    total_F /= count
    print(f'Total R: {total_R}')
    print(f'Total P: {total_P}')
    print(f'Total F: {total_F}')

In [4]:
estimate_chords(sampler, model, device, batch_size=8)

Eval: (N=3856, TP=2594, FP=1262, FN=86, P=0.673, R=0.968, F=0.794) - Greatest_Hits_I/01 Bohemian Rhapsody
Eval: (N=2337, TP=2107, FP=230, FN=30, P=0.902, R=0.986, F=0.942) - Greatest_Hits_I/02 Another One Bites The Dust
Eval: (N=2200, TP=2012, FP=188, FN=102, P=0.915, R=0.952, F=0.933) - Greatest_Hits_I/04 Fat Bottomed Girls
Eval: (N=1979, TP=1495, FP=484, FN=47, P=0.755, R=0.970, F=0.849) - Greatest_Hits_I/05 Bicycle Race
Eval: (N=1855, TP=1216, FP=639, FN=37, P=0.656, R=0.970, F=0.782) - Greatest_Hits_I/06 You're My Best Friend
Eval: (N=2280, TP=1263, FP=1017, FN=25, P=0.554, R=0.981, F=0.708) - Greatest_Hits_I/07 Don't Stop Me Now
Eval: (N=2466, TP=1858, FP=608, FN=100, P=0.753, R=0.949, F=0.840) - Greatest_Hits_I/08 Save Me
Eval: (N=1760, TP=1632, FP=128, FN=128, P=0.927, R=0.927, F=0.927) - Greatest_Hits_I/09 Crazy Little Thing Called Love
Eval: (N=3205, TP=1951, FP=1254, FN=80, P=0.609, R=0.961, F=0.745) - Greatest_Hits_I/10 Somebody To Love
Eval: (N=1894, TP=1366, FP=528, FN=50,

Eval: (N=197, TP=133, FP=64, FN=17, P=0.675, R=0.887, F=0.767) - Let_It_Be/05-Dig_It
Eval: (N=949, TP=812, FP=137, FN=30, P=0.856, R=0.964, F=0.907) - Let_It_Be/06-Let_It_Be
Eval: (N=171, TP=155, FP=16, FN=10, P=0.906, R=0.939, F=0.923) - Let_It_Be/07-Maggie_Mae
Eval: (N=861, TP=515, FP=346, FN=28, P=0.598, R=0.948, F=0.734) - Let_It_Be/08-I_ve_Got_A_Feeling
Eval: (N=686, TP=629, FP=57, FN=36, P=0.917, R=0.946, F=0.931) - Let_It_Be/09-One_After_909
Eval: (N=852, TP=592, FP=260, FN=22, P=0.695, R=0.964, F=0.808) - Let_It_Be/10-The_Long_And_Winding_Road
Eval: (N=609, TP=385, FP=224, FN=20, P=0.632, R=0.951, F=0.759) - Let_It_Be/11-For_You_Blue
Eval: (N=757, TP=540, FP=217, FN=11, P=0.713, R=0.980, F=0.826) - Let_It_Be/12-Get_Back
Eval: (N=678, TP=552, FP=126, FN=12, P=0.814, R=0.979, F=0.889) - Magical_Mystery_Tour/01-Magical_Mystery_Tour
Eval: (N=709, TP=255, FP=454, FN=4, P=0.360, R=0.985, F=0.527) - Magical_Mystery_Tour/02-The_Fool_On_The_Hill
Eval: (N=537, TP=400, FP=137, FN=12, P=0.

Eval: (N=758, TP=650, FP=108, FN=69, P=0.858, R=0.904, F=0.880) - The_White_Album_Disc_1/06-The_Continuing_Story_Of_Bungalow_Bill
Eval: (N=1107, TP=787, FP=320, FN=59, P=0.711, R=0.930, F=0.806) - The_White_Album_Disc_1/07-While_My_Guitar_Gently_Weeps
Eval: (N=640, TP=488, FP=152, FN=16, P=0.762, R=0.968, F=0.853) - The_White_Album_Disc_1/08-Happiness_Is_A_Warm_Gun
Eval: (N=589, TP=214, FP=375, FN=16, P=0.363, R=0.930, F=0.523) - The_White_Album_Disc_1/09-Martha_My_Dear
Eval: (N=488, TP=441, FP=47, FN=21, P=0.904, R=0.955, F=0.928) - The_White_Album_Disc_1/10-I_m_So_Tired
Eval: (N=542, TP=288, FP=254, FN=25, P=0.531, R=0.920, F=0.674) - The_White_Album_Disc_1/11-Blackbird
Eval: (N=485, TP=431, FP=54, FN=23, P=0.889, R=0.949, F=0.918) - The_White_Album_Disc_1/12-Piggies
Eval: (N=836, TP=225, FP=611, FN=3, P=0.269, R=0.987, F=0.423) - The_White_Album_Disc_1/13-Rocky_Raccoon
Eval: (N=902, TP=858, FP=44, FN=17, P=0.951, R=0.981, F=0.966) - The_White_Album_Disc_1/14-Don_t_Pass_Me_By
Eval: (